In [1]:
import argparse
import pathlib
import random

import numpy as np
import pandas as pd
import toml
from copairs import map
from copairs.matching import assign_reference_index

# check if in a jupyter notebook
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [2]:
if not in_notebook:
    parser = argparse.ArgumentParser(description="Match pairs of samples")
    parser.add_argument("--shuffle", action="store_true", help="Shuffle the data")

    args = parser.parse_args()
    shuffle = args.shuffle
else:
    shuffle = True

In [3]:
# load in the treatment groups
ground_truth = pathlib.Path(
    "../../4.sc_Morphology_Neural_Network_MLP_Model/MLP_utils/ground_truth.toml"
).resolve(strict=True)
# load in the ground truth
ground_truth = toml.load(ground_truth)
apoptosis_ground_truth = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis_ground_truth = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control_ground_truth = ground_truth["Healthy"]["healthy_groups_list"]

map_out_dir = pathlib.Path("../data/processed/mAP_scores/morphology/")
map_out_dir.mkdir(exist_ok=True, parents=True)

In [4]:
agg_data = pathlib.Path(
    "../../data/PBMC_preprocessed_sc_norm_aggregated.parquet"
).resolve(strict=True)
df = pd.read_parquet(agg_data)
# rename oneb_Metadata_Treatment_Dose_Inhibitor_Dose to Metadata_Treatment
df = df.rename(
    columns={"oneb_Metadata_Treatment_Dose_Inhibitor_Dose": "Metadata_Treatment"}
)
df.head()

,Metadata_Well,Metadata_Treatment,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,...,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_CorrMito_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_03_256,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrPM_3_03_256,Nuclei_Texture_Variance_CorrER_3_02_256,Nuclei_Texture_Variance_CorrMito_3_01_256
0,B02,LPS_0.010_ug_per_ml_DMSO_0.025_%,0.100173,-0.059734,0.218567,0.111938,0.007420,-0.100946,-0.030356,-0.070701,...,0.021386,-0.095924,-0.182695,-0.185317,-0.183084,-0.189434,0.217271,0.023909,-0.015452,-0.004886
1,B03,LPS_0.010_ug_per_ml_DMSO_0.025_%,0.137279,-0.097646,0.205644,0.108021,-0.002159,-0.141895,-0.059932,-0.091195,...,0.034647,0.079415,-0.105950,-0.112622,-0.108821,-0.114137,0.141156,0.022128,-0.017276,-0.006272
2,B04,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,0.071345,-0.053566,0.055404,0.013373,0.004443,-0.111708,-0.084402,-0.043409,...,-0.087337,-0.671670,-0.068129,-0.062520,-0.063204,-0.066542,0.074449,-0.020061,0.022286,0.039616
3,B05,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,0.110685,-0.084346,0.107954,0.071923,0.004150,-0.121376,-0.075382,-0.052805,...,-0.096255,-1.263923,-0.102173,-0.099375,-0.101330,-0.100625,0.114060,-0.007227,0.009458,0.059863
4,B06,DMSO_0.100_%_DMSO_0.025_%,-0.021771,0.018442,-0.048689,-0.070490,-0.005284,-0.008255,-0.012815,-0.017174,...,0.082642,0.292318,0.029805,0.022969,0.026496,0.024827,-0.028355,-0.007840,-0.037983,-0.014871


In [5]:
# add apoptosis, pyroptosis and healthy columns to dataframe
df["Apoptosis"] = df.apply(
    lambda row: row["Metadata_Treatment"] in apoptosis_ground_truth,
    axis=1,
)
df["Pyroptosis"] = df.apply(
    lambda row: row["Metadata_Treatment"] in pyroptosis_ground_truth,
    axis=1,
)
df["Control"] = df.apply(
    lambda row: row["Metadata_Treatment"] in control_ground_truth,
    axis=1,
)

# merge apoptosis, pyroptosis, and healthy columns into one column
df["Metadata_labels"] = df.apply(
    lambda row: "Apoptosis"
    if row["Apoptosis"]
    else "Pyroptosis"
    if row["Pyroptosis"]
    else "Control",
    axis=1,
)
metadata_labels = df.pop("Metadata_labels")
df.insert(1, "Metadata_labels", metadata_labels)
# # drop apoptosis, pyroptosis, and healthy columns
df.drop(columns=["Apoptosis", "Pyroptosis", "Control"], inplace=True)

In [6]:
if shuffle:
    random.seed(0)
    # permutate the data
    for col in df.columns:
        df[col] = np.random.permutation(df[col])

In [7]:
reference_col = "Metadata_reference_index"
df_activity = assign_reference_index(
    df,
    "Metadata_Treatment == 'DMSO_0.100_%_DMSO_0.025_%'",
    reference_col=reference_col,
    default_value=-1,
)
df_activity.head()

,Metadata_Well,Metadata_labels,Metadata_Treatment,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,...,Nuclei_Texture_InverseDifferenceMoment_CorrMito_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_03_256,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrPM_3_03_256,Nuclei_Texture_Variance_CorrER_3_02_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Metadata_reference_index
0,H10,Control,H2O2_100.000_uM_DMSO_0.025_%,0.026987,-0.052649,0.278934,0.017012,-0.008256,0.029140,-0.051623,...,-0.671670,-0.084725,-0.017563,-0.045506,0.009961,-0.066353,0.049471,-0.060734,-0.009392,-1
1,O03,Control,Flagellin_0.100_ug_per_ml_DMSO_0.025_%,-0.007621,-0.122214,0.137034,0.125031,0.003256,-0.111930,0.003535,...,-1.756236,0.158413,0.080793,-0.018635,-0.077171,0.003197,0.068586,-0.003248,-0.003802,-1
2,J12,Pyroptosis,LPS_0.010_ug_per_ml_DMSO_0.025_%,0.163241,0.013962,-0.049250,-0.089441,-0.002544,-0.101769,-0.049342,...,0.274264,-0.007692,0.174170,-0.016990,-0.090007,0.010649,-0.051418,-0.018757,0.005552,-1
3,G11,Control,LPS_10.000_ug_per_ml_Disulfiram_1.000_uM,0.320112,-0.097646,-0.098434,0.339274,0.005517,-0.124587,-0.015302,...,-0.816914,0.062135,-0.016748,0.041446,-0.007601,-0.015404,0.018940,0.118186,0.027890,-1
4,I07,Apoptosis,DMSO_0.100_%_Z-VAD-FMK_30.000_uM,0.140159,0.095494,0.071582,0.022305,-0.001287,-0.121376,0.010683,...,-0.512971,-0.159153,0.104797,-0.182915,-0.006841,0.060640,0.004740,0.135775,0.019494,-1


In [8]:
pos_sameby = ["Metadata_Treatment", reference_col]
pos_diffby = []
neg_sameby = []
neg_diffby = ["Metadata_Treatment", reference_col]
metadata = df_activity.filter(regex="Metadata")
profiles = df_activity.filter(regex="^(?!Metadata)").values

activity_ap = map.average_precision(
    metadata, profiles, pos_sameby, pos_diffby, neg_sameby, neg_diffby
)

activity_ap = activity_ap.query("Metadata_Treatment != 'DMSO_0.100_%_DMSO_0.025_%'")
activity_ap.head()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/home/lippincm/miniforge3/envs/map/lib/python3.13/site-packages/copairs/compute.py:401: RuntimeWarning: invalid value encountered in divide
  ap_scores = np.add.reduceat(pr_k * rel_k_list, cutoffs) / num_pos


,Metadata_Well,Metadata_labels,Metadata_Treatment,Metadata_reference_index,n_pos_pairs,n_total_pairs,average_precision
0,H10,Control,H2O2_100.000_uM_DMSO_0.025_%,-1,3,11,0.278571
1,O03,Control,Flagellin_0.100_ug_per_ml_DMSO_0.025_%,-1,3,11,0.566667
2,J12,Pyroptosis,LPS_0.010_ug_per_ml_DMSO_0.025_%,-1,3,11,0.331650
3,G11,Control,LPS_10.000_ug_per_ml_Disulfiram_1.000_uM,-1,3,11,0.539683
4,I07,Apoptosis,DMSO_0.100_%_Z-VAD-FMK_30.000_uM,-1,3,11,0.324242


In [9]:
activity_map = map.mean_average_precision(
    activity_ap, pos_sameby, null_size=1000000, threshold=0.05, seed=0
)
activity_map["-log10(p-value)"] = -activity_map["corrected_p_value"].apply(np.log10)
# flatten the multi-index columns to make it easier to work with
activity_map.reset_index(inplace=True)
# add apoptosis, pyroptosis and healthy columns to dataframe
activity_map["Apoptosis"] = activity_map.apply(
    lambda row: row["Metadata_Treatment"] in apoptosis_ground_truth,
    axis=1,
)
activity_map["Pyroptosis"] = activity_map.apply(
    lambda row: row["Metadata_Treatment"] in pyroptosis_ground_truth,
    axis=1,
)
activity_map["Control"] = activity_map.apply(
    lambda row: row["Metadata_Treatment"] in control_ground_truth,
    axis=1,
)

# merge apoptosis, pyroptosis, and healthy columns into one column
activity_map["Metadata_labels"] = activity_map.apply(
    lambda row: "Apoptosis"
    if row["Apoptosis"]
    else "Pyroptosis"
    if row["Pyroptosis"]
    else "Control",
    axis=1,
)
metadata_labels = activity_map.pop("Metadata_labels")
activity_map.insert(1, "Metadata_labels", metadata_labels)
# # drop apoptosis, pyroptosis, and healthy columns
activity_map.drop(columns=["Apoptosis", "Pyroptosis", "Control"], inplace=True)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

In [10]:
if shuffle:
    activity_map.to_parquet(map_out_dir / "activity_map_shuffled.parquet")
else:
    activity_map.to_parquet(map_out_dir / "activity_map.parquet")